In [1]:
import pandas as pd
import pyodbc
import paramiko
import numpy as np

C:\Users\ywang\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
# Change the directory to where the text file which stores vertica password is

pwd = open('..//..//vertica-pwd.txt','r').read()

## New Sub to connect exsisting ALLLLLLL ADVs

In [ ]:
## Get the information for indexing from the Ticket and manually update it 
para = {'pub_id':[12903], 
        'ads': {},
        'sub1': ['ulta','etsy','underarmour','bloomingdales','athleta','reebok','chewy','thebodyshop','michaelkors','macys','nike'],
        'sub2': ['shopping'],
        'sub3': ['dynamiclink'],
        'sub4': [''],
        'account_id': ['74843','74567','74609','74022','74875','73686','74319','74867','73804','8654','74521'],
        'ts_level_to-copy-ad':3,
        'ts_level_to-get-ts':3,
        'sub-value-to-copy-ad':'sub1',
        'sub-value-to-pass-ad':'sub1',        
       'ts_index':'TS_index_nateals_newadvs_setup_11102022.log',
       'cluster_sql':'./clusters/nateals_newadvs_setup_11102022.sql'
       }

In [ ]:
## Get the assets ready for direct match/manually update
para['ads'] = {'Ulta': [{'ADGROUP_ID': 15411303, 'KEYWORD_ID': 353493583}],
               'Etsy': [{'ADGROUP_ID': 14885621, 'KEYWORD_ID': 343200339}],
               'Under Armour': [{'ADGROUP_ID': 14894795, 'KEYWORD_ID': 346260429}],
               'Bloomingdales': [{'ADGROUP_ID': 15411653, 'KEYWORD_ID': 352243885}],
               'Athleta': [{'ADGROUP_ID': 15416354, 'KEYWORD_ID': 353225226}],
               'Reebok': [{'ADGROUP_ID': 14859161, 'KEYWORD_ID': 340076309}],
               'Chewy': [{'ADGROUP_ID': 15412735, 'KEYWORD_ID': 353019311}],
               'The Body Shop': [{'ADGROUP_ID': 15411545, 'KEYWORD_ID': 351482903}],
               'Michael Kors': [{'ADGROUP_ID': 15416194, 'KEYWORD_ID': 353362975}],
               'Macys': [{'ADGROUP_ID': 14966849, 'KEYWORD_ID': 350268077}],
               'Nike': [{'ADGROUP_ID': 14785161, 'KEYWORD_ID': 339354625}]
#                         {'ADGROUP_ID': 14966859, 'KEYWORD_ID': 347573853}]
#                  'accessories': [{'ADGROUP_ID': 14882321, 'KEYWORD_ID': 343090897}],
#                  'mattresses': [{'ADGROUP_ID': 14882319, 'KEYWORD_ID': 343090901}]
              }

In [ ]:
##Connect to the Database to read the traffic source and cluster ID
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;PWD=UID=ywang;PWD=ChangeMe!')
# Need to check the TS level to connect cluster / one ADV may have different campaign on different TS (e.g. DL Vs Homepage)
sql = """SELECT DISTINCT 
to_char({0}) as sub,
 -- {1},
-- lower(aa.external_name) as sub,
da.ADGROUP_ID, da.KEYWORD_ID 
    FROM ampx.traffic_source ats 
    JOIN domainpub.TS_CLUSTER ts ON ats.ID = ts.PUB_ATS_ID
    JOIN domainpub.ADGROUP_DM_CLUSTER da on da.DM_CLUSTER_ID = ts.CLUSTER_ID AND da.dm_cluster_id > 2000000000
    JOIN ampx.ADGROUP ad on ad.id = da.ADGROUP_ID
    LEFT JOIN ampx.KEYWORD kw on kw.id = da.KEYWORD_ID
    JOIN ampx.CAMPAIGN ac on ac.id = ad.CAMPAIGN_ID
    JOIN ampx.ACCOUNT aa on aa.id = ad.account_id
    where ats.publisher_id in ({2}) and ats.level = {3}
    --and sub2 like '%.uk%'
    --and aa.account_status_id = 1
    --and ac.campaign_status_id = 1
    --and ad.adgroup_status_id = 1
    --and kw.keyword_status_id = 1
    --and sub4 != 'google'
    --and sub2 like 'samsung%'
    --and aa.id != 74605
    order by 1
            """.format(para['sub-value-to-copy-ad'].replace(',','||'),para['sub-value-to-copy-ad'],str(para['pub_id'])[1:-1],para['ts_level_to-copy-ad'])
                       #str(para['account_id'])[1:-1])

ads = pd.read_sql(sql,db)


db.close()

In [ ]:
ads

In [ ]:
 ads['subs'] = ads['sub2'] +ads['sub3']+ [str(x).replace('None','') for x in ads['sub4']]

In [ ]:
ads['sub'] = [str(x)+str(y).replace('None','') for x, y in zip(ads['sub2'],ads['sub3'])]

In [ ]:
# ads.to_csv('avast_omnibox_assets.csv')

In [ ]:
# ads = pd.read_csv('avast_omnibox_assets.csv')

In [ ]:
lst = list(set(zip(ads['sub2'],ads['sub3'])))

In [ ]:
lst

In [ ]:
len(ads['sub'].unique())

In [ ]:
para['sub2']=ads['sub2'].unique()

In [ ]:
# para['sub1']=list(ads['sub1'].unique())
# para['sub3']=list(ads['sub3'].unique())
# para['sub4']=list(ads['sub4'].unique())


In [ ]:
file = open(para['ts_index'],'w')

# for i in ads['sub1'] : #,ads['sub2']):
#     line = str(para['pub_id'][0]) + '|' + str(i) + '|' +str(para['sub2'][0]) # + '|' +  str(j)+ '|'
#     print(line)
#     file.writelines(line+'\n')
# file.close()
# print('-'*80+'\n')

# for i,j in zip(ads[['sub2','sub3']].drop_duplicates()['sub2'],ads[['sub2','sub3']].drop_duplicates()['sub3']):
#     line = str(para['pub_id'][0]) + '|' + str(para['sub1'][0]) + '|' + str(i) + '|' + str(j).replace('None','')+ '|'
#     print(line)
#     file.writelines(line+'\n')
# file.close()

for i in para['sub1'] : #,ads['sub2']):
    for j in para ['sub2']:
        for k in para['sub3']:
            for l in para['sub4']:
                line = str(para['pub_id'][0]) + '|' + str(i) + '|' +str(j)  + '|' +  str(k)+ '|'+str(l)
                print(line)
                file.writelines(line+'\n')
file.close()

# for sub1 in para['sub1']:
#     for i,j in lst : #,ads['sub2']):
#         line = str(para['pub_id'][0])+ '|' + str(sub1) + '|' + str(i) + '|' +str(j)  #+ '|' +  str(k).replace('None','')
#         print(line)
#         file.writelines(line+'\n')
# file.close()
print('-'*80+'\n')

In [ ]:
ads = ads[['sub','ADGROUP_ID','KEYWORD_ID']].set_index('sub')

In [ ]:
ads

In [ ]:
for i in ads.index.unique().tolist():
#     print(i)
    para['ads'][i] = ads[ads.index == i].to_dict('records')

In [ ]:
para['ads']

In [ ]:
 para['sub2'] = [str(x) for x in para['sub2']]

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;PWD=UID=ywang;PWD=ChangeMe!')

sql = """SELECT DISTINCT to_char({0}) as sub,
ats.ID FROM ampx.TRAFFIC_SOURCE ats 
WHERE ats.publisher_id IN ({1}) 
and date(ats.CREATED_TIME_STAMP) = current_date()
and ats.sub1 IN ({2}) 
and ats.sub2 in ({3})
and level =3
-- and ((ats."LEVEL" = 3 and ats.id not in (select distinct TS_PARENT_ID_SUB3 from ampx.TRAFFIC_SOURCE where PUBLISHER_ID = 12751 and date(CREATED_TIME_STAMP) = current_date )) or ats.level=4) 
order by 1
""".format(para['sub-value-to-pass-ad'].replace(',','||'), str(para['pub_id'])[1:-1],str(para['sub1'])[1:-1], str(para['sub2'])[1:-1],para['ts_level_to-get-ts'])


# print(sql)

ats_id = pd.read_sql(sql,db)

# ats_id.set_index('ID',inplace=True)

# para['ats_id'] = ats_id.to_dict('index')

# print(len(para['ats_id']))

db.close()

In [ ]:
str(para['sub1'])

In [ ]:
ats_id

In [ ]:
ats_id['subs']  = [str(x)+str(y) for x, y in zip(ats_id['sub2'],ats_id['sub'])]

In [ ]:
ats_id

In [ ]:
ats_id.set_index('ID',inplace=True)

para['ats_id'] = ats_id[['sub']].to_dict('index')

In [ ]:
para['ats_id']

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;;PWD=UID=ywang;PWD=ChangeMe!')

sql = """SELECT max(ID)+1 from domainpub.DM_CLUSTER"""

cluster_1st = db.cursor().execute(sql).fetchall()[0][0]

db.close()

In [ ]:
cluster_1st
# cluster_1st = 2001371705

In [ ]:
para['cluster_sql'] 

In [ ]:
# Set up dynamic and direct matches
file = open(para['cluster_sql'],'w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for k in list(para['ats_id'].keys()):
    i = para['ats_id'][k]['sub']
    file.writelines('INSERT INTO DM_CLUSTER (ID, RESTRICTION_TYPE_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({}, 3, now(), now());'.format(cluster_1st))
    file.writelines('\n')

    file.writelines('INSERT INTO TS_CLUSTER (PUB_ATS_ID, CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({0}, {1}, now(), now());'.format(k,cluster_1st))
    file.writelines('\n')
     
    for j in para['ads'][i]:      
#         print(j)
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(j['ADGROUP_ID'],cluster_1st,j['KEYWORD_ID']))
        file.writelines('\n')
    file.writelines('\n\n')       
    
    cluster_1st += 1
                    
#     break
    
file.close()

## New ADV on One or more Pubs

In [2]:
para = {'pub_id':[12755], 
        'ads': {},
        'sub1': ['yahoomail'],
        'sub2': ['lookfantastic.com','myprotein.com'],
        'sub3': ['1','2','3','4'],
        'sub4': ['desktop','mobile'],
        'account_id': [],
        'ts_level_to-copy-ad':4,
        'ts_level_to-get-ts':4,
        'sub-value-to-copy-ad':'sub1',
        'sub-value-to-pass-ad':'sub1',        
       'ts_index':'TS_yahoomail_advs_index_11192024.txt',
       'cluster_sql':'./clusters/yahoomail_advs_setup_11192024.sql'
       }

In [ ]:
para = {'pub_id':[12285], 
        'ads': {},
        'sub1': ['thecontainerstore','tommybahama','ubereats','dermstore','everlane','hotels'],
        'sub2': ['slickdeals'],
        'sub3': ['dynamiclink'],
        'sub4': [''],
        'account_id': [],
        'ts_level_to-copy-ad':3,
        'ts_level_to-get-ts':3,
        'sub-value-to-copy-ad':'sub1',
        'sub-value-to-pass-ad':'sub1' ,       
       'ts_index':'slickdeals_advs_02222024_index.txt',
       'cluster_sql':'./clusters/slickdeals_advs_02222024_setup.sql'
       }

In [7]:
para = {'pub_id':[13083], 
        'ads': {},
        'sub1': ['web','app'],
        'sub2': ['amazon','bestbuy','homedepot','ikea','tmobile','wayfair','ulta','nike','officedepot','macys','chewy','bedbathbeyond','thecontainerstore','forever21','fragrancenet','underarmour','saksoff5th','bloomingdales','newbalance','michaelkors','jcpenny','tommybahama','calvinklein','spanx','harrydavid','oldnavy','overstock','reebok','victoriasecret','1800flowers','rosssimons','1800contacts'],
        'sub3': [''],
        'sub4': [''],
        'account_id': [],
        'ts_level_to-copy-ad':2,
        'ts_level_to-get-ts':2,
        'sub-value-to-copy-ad':'sub2',
        'sub-value-to-pass-ad':'sub2' ,       
       'ts_index':'vetted_10292024_index.txt',
       'cluster_sql':'./clusters/reviewed_newsub3_06202024_setup.sql'
       }

In [ ]:
para = {'pub_id':[12833], 
        'ads': {},
        'sub1': ['wired.com','latimes.com','usnews.com','gq.com','wsj.com','washingtonpost.com','telegraph.co.uk'],
        'sub2': ['tracking.feedmob.com'],
        'sub3': ['homepage','dynamiclink'],
        'sub4': [''],
        'account_id': [],
        'ts_level_to-copy-ad':3,
        'ts_level_to-get-ts':3,
        'sub-value-to-copy-ad':'sub2',
        'sub-value-to-pass-ad':'sub2' ,       
       'ts_index':'SU_uber_02222024_index.txt',
       'cluster_sql':'./clusters/SU_uber_02222024_setup.sql'
       }

In [ ]:
para = {'pub_id':[12593], 
        'ads': {},
        'sub1': ['mercedes'],
        'sub2': ['carrier'],
        'sub3': ['AT1lhrbw0yibonid4nvyd2fd0gb'],
        'sub4': ['push'],
        'account_id': [],
        'ts_level_to-copy-ad':4,
        'ts_level_to-get-ts':4,
        'sub-value-to-copy-ad':'sub1',
        'sub-value-to-pass-ad':'sub1',        
       'ts_index':'apptap_mercedes_index_11142023.txt',
       'cluster_sql':'./clusters/apptap_mercedes_setup_11142023.sql'
       }

In [ ]:
para['ads'] = [
        {'ADGROUP_ID': 15482791, 'KEYWORD_ID': 357840725},
#         {'ADGROUP_ID': 15482745, 'KEYWORD_ID': 355642451},
#         {'ADGROUP_ID': 15482641, 'KEYWORD_ID': 355364555},
#         {'ADGROUP_ID': 15482633, 'KEYWORD_ID': 355366699},
#         {'ADGROUP_ID': 15482173, 'KEYWORD_ID': 355164951},
#         {'ADGROUP_ID': 15411545, 'KEYWORD_ID': 351482903},
#         {'ADGROUP_ID': 15416354, 'KEYWORD_ID': 353225226},
#         {'ADGROUP_ID': 15229609, 'KEYWORD_ID': 353367393},
#         {'ADGROUP_ID': 14959739, 'KEYWORD_ID': 350512671},
#         {'ADGROUP_ID': 15416793, 'KEYWORD_ID': 353368889},
#         {'ADGROUP_ID': 10929054, 'KEYWORD_ID': 285129392},
#         {'ADGROUP_ID': 14859145, 'KEYWORD_ID': 342721341},
#         {'ADGROUP_ID': 15477007, 'KEYWORD_ID': 353942671},
#         {'ADGROUP_ID': 15416240, 'KEYWORD_ID': 353208680},
#         {'ADGROUP_ID': 15411629, 'KEYWORD_ID': 351512915},
#         {'ADGROUP_ID': 15411313, 'KEYWORD_ID': 353659041},
#         {'ADGROUP_ID': 15412805, 'KEYWORD_ID': 353367389},
#         {'ADGROUP_ID': 15478441, 'KEYWORD_ID': 353988001},
#         {'ADGROUP_ID': 15481481, 'KEYWORD_ID': 354137077},
#         {'ADGROUP_ID': 15480533, 'KEYWORD_ID': 354119383},
#         {'ADGROUP_ID': 15481477, 'KEYWORD_ID': 354137073},
#         {'ADGROUP_ID': 15480529, 'KEYWORD_ID': 354119381}
              ]

In [ ]:
## Getting all existing subs or placements from records
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """SELECT DISTINCT 
to_char({0}) as sub
-- lower(aa.external_name) as sub,
    FROM ampx.traffic_source ats 
    JOIN domainpub.TS_CLUSTER ts ON ats.ID = ts.PUB_ATS_ID
    JOIN domainpub.ADGROUP_DM_CLUSTER da on da.DM_CLUSTER_ID = ts.CLUSTER_ID AND da.dm_cluster_id > 2000000000
    JOIN ampx.ADGROUP ad on ad.id = da.ADGROUP_ID
    LEFT JOIN ampx.KEYWORD kw on kw.id = da.KEYWORD_ID
    JOIN ampx.CAMPAIGN ac on ac.id = ad.CAMPAIGN_ID
    JOIN ampx.ACCOUNT aa on aa.id = ad.account_id
    where ats.publisher_id in ({1}) and ats.level = {2}
    --and sub2 not in ('test')
    --and sub1 not in ('hotukdeals','savoo')
    --and aa.account_status_id = 1
    --and ac.campaign_status_id = 1
    --and ad.adgroup_status_id = 1
    --and kw.keyword_status_id = 1
    --and sub1 in ('chollometro.savings-united.com','20minutos.es','descuentos.abc.es','chollometro.com')
    --and sub2 not in ('test','coupons.businessinsider.com')
    --and sub2 !='mediabuy'
    --and sub3 ='dynamiclink'
    order by 1
            """.format(para['sub-value-to-copy-ad'],str(para['pub_id'])[1:-1],para['ts_level_to-copy-ad'])
                       #str(para['account_id'])[1:-1])

ads = pd.read_sql(sql,db)


db.close()

In [ ]:
## Special Case
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """
SELECT distinct sub2 as sub
from ampx.TRAFFIC_SOURCE ts 
where ts.PUBLISHER_ID = 12235
and sub2 not in (SELECT distinct sub2
                from ampx.TRAFFIC_SOURCE ts  
                where ts.PUBLISHER_ID = 12235
                and sub1 = 'reebok'
                and "LEVEL" = 2)
and "LEVEL" = 2
"""
ads = pd.read_sql(sql,db)

db.close()

In [ ]:
## VL case
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """
SELECT distinct sub1
from ampx.TRAFFIC_SOURCE ts 
where ts.PUBLISHER_ID = 12359
and sub1 not in (select distinct ts.sub1 from ampx.TRAFFIC_SOURCE ts join ampx.GLOBAL_BLACKLIST gb on gb.TS_ID = ts.id )
and sub2 = 'wix'
and sub3 = 'homepage'
and LEVEL =3 
"""
ads = pd.read_sql(sql,db)

db.close()


In [ ]:
## VL case
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """
with a as (
SELECT DISTINCT 
	pub.id, pub.PUBLISHER_NAME,
	ats.sub1,ats.sub2,ats.sub3,
	da.ADGROUP_ID ,da.KEYWORD_ID ,ac.name, ac.CAMPAIGN_STATUS_ID , ad.ADGROUP_STATUS_ID , ke.KEYWORD_STATUS_ID,
		case when ke.DEST_URL like '%PATH%' then 'dynamic' else 'homepage' end as Asset_type
--ats.sub1,
--ats.sub2,ats.sub3,ad.CAMPAIGN_ID ,da.ADGROUP_ID ,da.KEYWORD_ID ,ac.CAMPAIGN_STATUS_ID , ad.ADGROUP_STATUS_ID , ke.KEYWORD_STATUS_ID --, ats.sub1,ats.sub2,ats.sub3, ats.CREATED_TIME_STAMP ,da.id,aa.NAME ,,da.CREATED_TIME_STAMP  
FROM ampx.TRAFFIC_SOURCE ats
JOIN domainpub.TS_CLUSTER ts on ats.id = ts.PUB_ATS_ID
JOIN domainpub.ADGROUP_DM_CLUSTER da on da.DM_CLUSTER_ID = ts.CLUSTER_ID and da.dm_cluster_id > 2000000000
JOIN ampx.ADGROUP ad  on ad.id = da.ADGROUP_ID
JOIN admindb_prod.PUBLISHER pub on pub.id = ats.PUBLISHER_ID
JOIN admindb_prod.PUB_DELIVERY_METHOD del on del.id = pub.PUB_DELIVERY_METHOD_ID
JOIN ampx.ACCOUNT aa on aa.id = ad.account_id
JOIN ampx.CAMPAIGN ac on ac.id = ad.CAMPAIGN_ID
LEFT JOIN ampx.KEYWORD ke on ke.id = da.KEYWORD_ID
--where (da.ADGROUP_ID = 14901191 and da.KEYWORD_ID = 346253805) 
--and da.id in (42149641,42149835,42149505,42149421,42149527,42149365,42149437,42149831,42149501,42149879,42149565,42149751,42149803,42149535,42149457,42149853,42149497,42149439,42149403,42149369,42149715,42149423,42149771,42149363,42149653,42149455,42149569,42149547,42149795,42149495,42149861,42149613,42149839,42149791,42149663,42149717,42149415,42149543,42149575,42149693,42149855,42149419,42149673,42149475,42149329,42149675,42149897,42149525,42149891,42149429,42149343,42149617,42149631,42149529,42149671,42149451,42149651,42149339,42149851,42149893,42149843,42149597,42149467,42149787,42149573,42149873,42149461,42149411,42149645,42149737,42149837,42149579,42149667,42149399,42149557,42149695,42149559,42149463,42149621,42149359,42149585,42149521,42149377,42149371,42149449,42149679,42149773,42149499,42149657,42149813,42149431,42149655,42149493,42149625,42149863,42149391,42149775,42149395,42149711,42149871,42149881,42149709,42149383,42149823,42149677,42149517,42149727,42149519,42149619,42149409,42149725,42149731,42149417,42149335,42149353,42149533,42149607,42149591,42149901,42149859,42149443,42149699,42149347,42149703,42149401,42149471,42149627,42149741,42149643,42149605,42149333,42149907,42149425,42149503,42149887,42149633,42149485,42149807,42149801,42149487,42149867,42149661,42149805,42149701,42149765,42149405,42149707,42149447,42149745,42149473,42149581,42149635,42149553,42149441,42149869,42149603,42149531,42149681,42149469,42149589,42149491,42149733,42149637,42149555,42149753,42149345,42149819,42149777,42149883,42149743,42149739,42149789,42149811,42149649,42149393,42149623,42149885,42149355,42149705,42149849,42149373,42149567,42149759,42149545,42149583,42149735,42149875,42149817,42149639,42149541,42149385,42149763,42149669,42149549,42149865,42149825,42149561,42149489,42149821,42149595,42149413,42149509,42149847,42149845,42149551,42149877,42149799,42149747,42149379,42149381,42149757,42149659,42149593,42149459,42149571,42149341,42149389,42149905,42149857,42149599,42149361,42149387,42149523,42149895,42149827,42149513,42149833,42149665,42149769,42149793,42149587,42149815,42149349,42149611,42149507,42149841,42149515,42149755,42149797,42149483,42149577,42149629,42149331,42149647,42149481,42149433,42149351,42149713,42149465,42149357,42149563,42149511,42149615,42149721,42149697,42149761,42149337,42149785,42149375,42149479,42149427,42149539,42149683,42149767,42149719,42149809,42149729,42149407,42149749,42149687,42149445,42149397,42149685,42149691,42149537,42149723,42149779,42149609,42149477,42149829,42149783,42149903,42149909,42149889,42149601,42149781,42149689,42149435,42149453,42149367,42149899)
where ad.CAMPAIGN_ID in (211337)--215015,212425) 
and ats.PUBLISHER_ID = 12359
and ats.sub1 not in (select DISTINCT ts.sub1
					from ampx.GLOBAL_BLACKLIST gb 
					join ampx.TRAFFIC_SOURCE ts on ts.id = gb.TS_ID
					where ts.PUBLISHER_ID = 12359)
--where pub.id in (12359)
--where da.id in (41366647,41366649,41366651)
--where aa.id = 74161
--and ke.DEST_URL like '%PATH%'
--and sub2 = 'verizon'
and pub.ACTIVE = 1
--and pub.ID not in (12125,12121)
)
SELECT distinct SUB1 
from ampx.TRAFFIC_SOURCE ts 
where ts.PUBLISHER_ID = 12359
and ts.sub1 not in (select DISTINCT ts.sub1
					from ampx.GLOBAL_BLACKLIST gb 
					join ampx.TRAFFIC_SOURCE ts on ts.id = gb.TS_ID
					where ts.PUBLISHER_ID = 12359)
and sub1 not in ('3998718','4926625','1534853','1623615','1643950','3201772','5208207','1643958','1621274','1528657','4139703','1621969','1623619','4926621','1528659','1706327','3989807','1704907','1567217','1610189','3885371','3997475','1519029','3989647','3601149','3722531','1608317','1637880','1962179','1698716','1303172','1573367','1650064','3251458','1604115','5114286','1529964','1769639','1601942','1527048','1596650','1300149','1595963','3351659','4961866','1883482','1676422','4439479','1659501','4283019','3606768','1676400','1758927','4013280','1918823','4867665','1926905','1534861')
and sub1 not in (select distinct sub1 from a)
"""
ads = pd.read_sql(sql,db)

db.close()


In [ ]:
ads

In [ ]:
# df = pd.read_excel('yieldkit_als_setup.xlsx',dtype=str)
# df.head()

In [ ]:
# df[4:]['adv']

In [ ]:
# df = pd.read_excel('yahoomail_als_setup.xlsx')
# para['sub2'] = list(set(df[4:]['adv'].to_list()))

In [ ]:
para['sub1'] = ads['sub'].to_list()

In [ ]:
para['sub2'] = ads['sub'].to_list()

In [3]:
file = open(para['ts_index'],'w') 

for i in para['pub_id']:
    for j in para['sub1']:
            for k in para['sub2']:
                    for l in para['sub3']:
                            for m in para['sub4']:
                                line = str(i) + '|' + str(j) + '|' + str(k) + '|' + str(l) + '|' + str(m)                           
                                file.writelines('%s\n'%line)
# print(i,j,k,l)
    
file.close()

In [ ]:
for i,j in list(zip(ads['sub2'],ads['sub3'])):
    print(i,j)

In [ ]:
file = open(para['ts_index'],'w') 

for i in para['pub_id']:
    for j in para['sub1']:
            for k,l in list(zip(ads['sub2'],ads['sub3'])):
                    for m in para['sub4']:
                        line = str(i) + '|' + str(j) + '|' + str(k)+ '|' + str(l) + '|' + str(m)                           
                        file.writelines('%s\n'%line)

    
file.close()

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """SELECT DISTINCT {0} as sub,
ats.ID FROM ampx.TRAFFIC_SOURCE ats 
WHERE ats.publisher_id IN ({1}) 
and date(ats.CREATED_TIME_STAMP) = current_date () 
--and ats.sub1 IN ({2}) 
--and ats.sub2 in ({3})
--and ats.sub3 in ('19')
--and ats.sub3 = 'dynamiclink'
--and ats.sub2 = 'tui'
--and level = 3
and level = ({4})
--order by 1
""".format(para['sub-value-to-pass-ad'].replace(',','||'),str(para['pub_id'])[1:-1],str(para['sub1'])[1:-1],
           str(para['sub2'])[1:-1],para['ts_level_to-get-ts'])
ats_id = pd.read_sql(sql,db)

para['ats_id'] = ats_id['ID'].to_list()
para['ats_id'].sort()
print(len(para['ats_id']))
db.close()

In [ ]:
list(ats_id.ID).sort()

In [ ]:
## Special Case
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """
SELECT distinct id, sub1 as sub1, sub4 as sub4
from ampx.TRAFFIC_SOURCE ts 
where ts.PUBLISHER_ID = 12751
--and sub1 = 'bcbswashington'
--and sub4 = 'display'
and sub3 = 'fr'
and sub2 = 'thebodyshop'
""".format(para['sub-value-to-pass-ad'].replace(',','||'),str(para['pub_id'])[1:-1],str(para['sub1'])[1:-1],
           str(para['sub2'])[1:-1],para['ts_level_to-get-ts'])
ats_id = pd.read_sql(sql,db)

para['ats_id'] = ats_id['id'].to_list()
print(len(para['ats_id']))

db.close()

In [ ]:
# ats_id = ats_id.set_index('sub')
lst = ats_id.to_dict('i')

In [ ]:
df = pd.read_excel('yieldkit_als_setup.xlsx',dtype=str)

In [ ]:
df[4:]

In [ ]:
df_new = df[4:][['adv','Adgroup ID','Keyword ID']].set_index('adv')

In [ ]:
# dic = df[['sub2','Adgroup_ID','Keyword_ID']].set_index('sub2')

dic = {}
for i in df[4:]["adv"].tolist(): 
    dic[i] = list(zip(df[df["adv"]==i]["Adgroup ID"],df[df["adv"]==i]["Keyword ID"]))

In [ ]:
lst

In [ ]:
dic[lst[1]['sub']]

In [ ]:
para['ats_id']

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """SELECT max(ID)+1 from domainpub.DM_CLUSTER"""

cluster_1st = db.cursor().execute(sql).fetchall()[0][0]


db.close()

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')
sql = """
select distinct publisher_id, sub1, sub2, sub3 from ampx.traffic_source ats 
JOIN domainpub.TS_CLUSTER ts ON ats.ID = ts.PUB_ATS_ID 
where 
--ats.publisher_id in (%s) and level = 2
ats.publisher_id = 12513
and sub1 in ('1525083','1598242')
--and sub2 = 'thebodyshop'
and sub2 = '74437'
--and sub3= 'dynamiclink'
--and sub1 in ('10075')
--and ats.PUBLISHER_ID||sub2 not in (select distinct publisher_id||sub2 from ampx.traffic_source ats where sub3 = 73812 and level =3 and PUBLISHER_ID in (12379,12361,12395,12339))
order by 1,2,3
"""%str(para['pub_id'])[1:-1]

sub2 = pd.read_sql(sql,db)

para['sub2'] = sub2.to_dict('records')

##para['tag_id'] = sub2[['publisher_id','sub1']].drop_duplicates().to_dict('records')

sub2 = pd.read_sql(sql,db)

sql = """SELECT DISTINCT ts.CLUSTER_ID FROM ampx.traffic_source ats 
            JOIN domainpub.TS_CLUSTER ts ON ats.ID = ts.PUB_ATS_ID
                where  ats.publisher_id = 12513 and ats.level = 2
                --and sub1 = 'yelp'
                and sub1 in ('1525083','1598242')
                --ats.publisher_id in (%s) and ats.level = 2
                and sub2 = '74437'
                --and sub2 = 'filetaxes'
                --and sub3 = 'gb'
                --and sub1 in ('10075')
                --and sub2 not in (select distinct sub2
                  --              from ampx.TRAFFIC_SOURCE ts 
                    --            join admindb_prod.PUBLISHER p on p.id = ts.PUBLISHER_ID 
                      --          where ts.PUBLISHER_ID = 12481
                        --        and sub3 = '74319')
                 --and ats.id not in (--SELECT distinct ts.id
                                    --from domainpub.TS_CLUSTER tc 
                                    --join ampx.TRAFFIC_SOURCE ts on ts.id = tc.PUB_ATS_ID 
                                    --join domainpub.ADGROUP_DM_CLUSTER adc on adc.DM_CLUSTER_ID = tc.CLUSTER_ID 
                                    --join ampx.ADGROUP ad on ad.id = adc.ADGROUP_ID 
                                    --join ampx.CAMPAIGN ac on ac.id = ad.CAMPAIGN_ID 
                                    --join admindb_prod.PUBLISHER p on p.id = ts.PUBLISHER_ID 
                                    --where ts.PUBLISHER_ID in (12411)
                                    --and ad.ACCOUNT_ID =74565
                                    --and level =2 )
                """%str(para['pub_id'])[1:-1]

para['clusters'] = pd.read_sql(sql,db)['CLUSTER_ID'].tolist()

db.close()

##if len(para['pub_id']) == len(para['tag_id']):
    ##print('Looks good. Move forward')
##else:
    ##print('Is there any special pubs (one with multiple tag iDs) included. If yes, need to take them out')
    
##print(len(para['tag_id']))
print(len(para['clusters']))

In [ ]:
# cluster_1st = 2001374496
cluster_1st

In [ ]:
# Set up dynamic and direct matches
file = open(para['cluster_sql'],'w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for i in para['ats_id'] :
    file.writelines('INSERT INTO DM_CLUSTER (ID, RESTRICTION_TYPE_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({}, 3, now(), now());'.format(cluster_1st))
    file.writelines('\n')

    file.writelines('INSERT INTO TS_CLUSTER (PUB_ATS_ID, CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({0}, {1}, now(), now());'.format(i,cluster_1st))
    file.writelines('\n')   
    for j in para['ads']:
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(j['ADGROUP_ID'],cluster_1st,j['KEYWORD_ID']))
        file.writelines('\n')
    file.writelines('\n\n') 
    cluster_1st += 1
                    
#     break
    
file.close()

In [ ]:
para['ads']

In [ ]:
# Set up dynamic and direct matches
file = open('skl_samsungcrn_setup_20221012.sql','w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for j in para['ads']:
    for i in range(int(para['clusters'])):
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(j['ADGROUP_ID'],para['clusters'][i],j['KEYWORD_ID']))
        file.writelines('\n')
        file.writelines('\n\n') 

#     break
    
file.close()

In [ ]:
# Set up dynamic and direct matches
file = open('yieldkit_hotels_setup_20220309.sql','w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for j in lst :
    file.writelines('INSERT INTO DM_CLUSTER (ID, RESTRICTION_TYPE_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({}, 3, now(), now());'.format(cluster_1st))
    file.writelines('\n')
    i=lst[j]
    file.writelines('INSERT INTO TS_CLUSTER (PUB_ATS_ID, CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({0}, {1}, now(), now());'.format(str(i['ID']),cluster_1st))
    file.writelines('\n')   

    file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(dic[i['sub']]['ADGROUP_ID'],cluster_1st,dic[i['sub']]['KEYWORD_ID']))
    file.writelines('\n')
    file.writelines('\n\n')       
    
    cluster_1st += 1
                    
#     break
    
file.close()

In [ ]:
dic[lst[0]['sub']][0]

In [ ]:
# Set up dynamic and direct matches
file = open(para['cluster_sql'],'w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for j in lst :
    file.writelines('INSERT INTO DM_CLUSTER (ID, RESTRICTION_TYPE_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({}, 3, now(), now());'.format(cluster))
    file.writelines('\n')
    i=lst[j]
    file.writelines('INSERT INTO TS_CLUSTER (PUB_ATS_ID, CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({0}, {1}, now(), now());'.format(str(i['ID']),cluster))
    file.writelines('\n')   

    for k in dic[i['sub']]:
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(k[0],cluster,k[1]))
        file.writelines('\n')
    file.writelines('\n\n')       
    
    cluster += 1
                    
#     break
    
file.close()

In [ ]:
lst

## ADV update asset

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """select distinct ts.PUBLISHER_ID ,ts.sub1,ts.sub2,ts.sub3, ts.id as ts_id, tc.CLUSTER_ID 
from ampx.TRAFFIC_SOURCE ts 
join domainpub.TS_CLUSTER tc on tc.PUB_ATS_ID = ts.ID 
join domainpub.ADGROUP_DM_CLUSTER adc on adc.DM_CLUSTER_ID = tc.CLUSTER_ID 
where PUBLISHER_ID in (12755)
--and ts.TS_PARENT_ID_SUB1 not in (select gb.TS_ID from ampx.GLOBAL_BLACKLIST gb )
and sub1 not in (SELECT distinct ats.SUB1 
                        from ampx.TRAFFIC_SOURCE ats
                        join ampx.GLOBAL_BLACKLIST bl on bl.ts_id = ats.ID 
                        where ats.PUBLISHER_ID in (12359,12371) and level = 1)
and level = 4
--and sub1 = '10156'
and sub2 = 'boots.com'
--and sub3 = '74433'
--and sub3 in ('1','2','3','4')
--and sub4 in ('desktop','mobile')
order by 1,2
"""

ats_id = pd.read_sql(sql,db)

cluster_id = ats_id['CLUSTER_ID'].to_list()

print(len(cluster_id))

db.close()

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """SELECT distinct p.id, p.PUBLISHER_NAME , ts.SUB1 , ts.SUB2 , tc.CLUSTER_ID -- adc.ADGROUP_ID 
from ampx.TRAFFIC_SOURCE ts
join admindb_prod.PUBLISHER p on p.id = ts.PUBLISHER_ID 
join domainpub.TS_CLUSTER tc on tc.PUB_ATS_ID = ts.TS_PARENT_ID_SUB2 
join domainpub.ADGROUP_DM_CLUSTER adc on adc.DM_CLUSTER_ID = tc.CLUSTER_ID 
join ampx.ADGROUP ad on ad.id =adc.ADGROUP_ID 
where ts.sub3 = '74183'
and p.ACTIVE = 1
and p.PUB_DELIVERY_METHOD_ID = 10
and ad.ACCOUNT_ID = 74183
and p.id not in (12481)
"""

ats_id = pd.read_sql(sql,db)

cluster_id = ats_id['CLUSTER_ID'].to_list()

print(len(cluster_id))

db.close()

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """SELECT distinct adc.ADGROUP_ID , adc.KEYWORD_ID , adc.DM_CLUSTER_ID 
from ampx.TRAFFIC_SOURCE ts
join domainpub.TS_CLUSTER tc on tc.PUB_ATS_ID = ts.id
join domainpub.ADGROUP_DM_CLUSTER adc on adc.DM_CLUSTER_ID = tc.CLUSTER_ID 
where ts.PUBLISHER_ID = 12845
and ts.sub2  = 'shopsponsoredbrands'
-- and sub1 not in (
                SELECT DISTINCT SUB1
                FROM ampx.TRAFFIC_SOURCE ats 
                JOIN ampx.GLOBAL_BLACKLIST bl on bl.TS_ID = ats.id
                where ats.PUBLISHER_ID in (12359,12371)
                and ats.level <= 2
                )
"""

ats_id = pd.read_sql(sql,db)

cluster_id = ats_id['DM_CLUSTER_ID'].to_list()

print(len(cluster_id))

db.close()

In [ ]:
cluster_id

### Updating cluster checking for Dynamic Tiles

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """select distinct ts.PUBLISHER_ID ,ts.sub1,ts.sub2,ts.sub3, ts.id as ts_id, tc.CLUSTER_ID, adc.ID as clusterid
from ampx.TRAFFIC_SOURCE ts 
join domainpub.TS_CLUSTER tc on tc.PUB_ATS_ID = ts.ID 
join domainpub.ADGROUP_DM_CLUSTER adc on adc.DM_CLUSTER_ID = tc.CLUSTER_ID 
join ampx.KEYWORD k on k.ID = adc.KEYWORD_ID
where PUBLISHER_ID in (122)
--and ts.TS_PARENT_ID_SUB1 not in (select gb.TS_ID from ampx.GLOBAL_BLACKLIST gb )
and sub1 not in (SELECT distinct ats.SUB1 
                        from ampx.TRAFFIC_SOURCE ats
                        join ampx.GLOBAL_BLACKLIST bl on bl.ts_id = ats.ID 
                        where ats.PUBLISHER_ID in (12845) and level = 1)
--and level = 2
and sub1 = 'samsung'
--and sub2 = 'shopsponsoredbrands'
--and sub3 = '74433'
--and sub3 in ('1','2','3','4')
--and sub4 in ('desktop','mobile')
--and adc.ADGROUP_ID = 15411333
--and k.ID = 351451423
order by 1,2
"""

ats_id = pd.read_sql(sql,db)

cluster_id = ats_id['CLUSTER_ID'].to_list()

id_of_cluster = ats_id['clusterid'].to_list()

print(len(cluster_id))

print(len(id_of_cluster))

db.close()

In [ ]:
cluster_id

In [ ]:
id_of_cluster

In [ ]:
df = pd.read_csv('wix_asset.csv')

In [ ]:
df = df.iloc[[18,]]

In [ ]:
df

In [ ]:
df[['ad group id','keyword id']]

In [ ]:
# Set up dynamic and direct matches
file = open('disneyplust_new_assets_20220414.sql','w') 

file.writelines('\nuse domainpub;\n')

for i in cluster_id :
    for j,k in zip(df['ad group id'],df['keyword id']):
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(j,i,k))
        file.writelines('\n')
    #file.writelines('\n\n')       
    
                    
#     break
    
file.close()

In [ ]:
ad= 15411333
kw = 351451569

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel('chewy new asset for insertation.xlsx')

In [ ]:
df.head()

In [ ]:
ad = '15412813'
kw = '303327629'
cluster_id = list(df.DM_CLUSTER_ID)

In [ ]:
# Set up dynamic and direct matches
file = open('update_samsung_slickdeals_20220803.sql','w') 

file.writelines('\nuse domainpub;\n')

file.writelines('update ADGROUP_DM_CLUSTER set ADGROUP_ID = 15411333, KEYWORD_ID = 351451569, MODIFIED_TIME_STAMP = now() 
                WHERE ID IN (42306901,42306905,42306927,42306929,42379625,42306935,42306939,42379639,42306911,42306907,42306909,42306903,42306919,42306941,42306915,42306925) AND ADGROUP_ID = 14960135 AND KEYWORD_ID = 346924963;')     
    
                    
#     break
    
file.close()

In [ ]:
#Update Assets without inserting TS into new assets, Ask for Permission before do UPDATING SQL
file = open('samsungmobile_new_assets_20220426.sql','w') 

file.writelines('\nuse domainpub;\n')

for i in cluster_id :
    file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(ad,i,kw))
    file.writelines('\n')
    #file.writelines('\n\n')       
    
                    
#     break
    
file.close()

## Insert New asset to exsisting cluster (Account Restructure)

In [ ]:
ads = [
        {'ADGROUP_ID': 12348661, 'KEYWORD_ID': 303327625},
#         {'ADGROUP_ID': 15622707, 'KEYWORD_ID': 357878017},
#         {'ADGROUP_ID': 15001135, 'KEYWORD_ID': 349175843},
#         {'ADGROUP_ID': 14959687, 'KEYWORD_ID': 346777953},
#         {'ADGROUP_ID': 15229049, 'KEYWORD_ID': 349959965},
#         {'ADGROUP_ID': 15229045, 'KEYWORD_ID': 349959961},
# ,{'ADGROUP_ID':15229055, 'KEYWORD_ID':349959971}
# ,{'ADGROUP_ID':15229047, 'KEYWORD_ID':349959963}
# ,{'ADGROUP_ID':15229057, 'KEYWORD_ID':349959973}
# ,{'ADGROUP_ID':15229043, 'KEYWORD_ID':349959959}
# ,{'ADGROUP_ID':15229051, 'KEYWORD_ID':349959967}
# ,{'ADGROUP_ID':15229053, 'KEYWORD_ID':349959969}
# ,{'ADGROUP_ID':15229041, 'KEYWORD_ID':349959957}
              ]

In [ ]:
ads

In [ ]:
# df = pd.read_excel('total_av_amply_cluster_id.xlsx')
df = pd.read_csv('avast_browser_tiles_new_advs.csv')

In [ ]:
df.head()

In [ ]:
# cluster_id = list(df['DM_CLUSTER_ID'])

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=ywang;PWD=ChangeMe!')

sql = """SELECT distinct adc.DM_CLUSTER_ID 
from ampx.TRAFFIC_SOURCE ts
Join admindb_prod.PUBLISHER pub on pub.id = ts.PUBLISHER_ID
join domainpub.TS_CLUSTER tc on tc.PUB_ATS_ID = ts.id
join domainpub.ADGROUP_DM_CLUSTER adc on adc.DM_CLUSTER_ID = tc.CLUSTER_ID 
Join ampx.ADGROUP ad  on ad.id = adc.ADGROUP_ID
Join ampx.ACCOUNT aa on aa.id = ad.account_id
where 1=1
and ts.PUBLISHER_ID in (12759,12487,12219)
--pub.PUB_DELIVERY_METHOD_ID = 3
--and ts.ID in (317847186)
--and aa.Name like '%Samsung%'
--and aa.id = 74949
--and ts.id in (318255897)
--and ts.sub1 in ('yahoomail')
--and ts.sub3 not in ('deeplink')
--and ts.sub2 in ('samsung.com')
--and ts.UI_SUB1_ID in (181,221,145,144,142,172)
--and ts.sub1 in ('samsung-best-android-tablets','samsung-best-budget-laptops','samsung-best-chromebooks','samsung-best-computer-monitors','samsung-best-fitness-trackers','samsung-best-frame-tvs','samsung-best-kids-tablets','samsung-best-laptop-computers','samsung-best-laptops-for-college-students','samsung-best-laptops-under-500','samsung-best-qled-tvs','samsung-best-samsung-galaxy-tablets','samsung-best-samsung-laptops','samsung-best-samsung-smart-watches','samsung-best-samsung-tablets','samsung-best-samsung-tvs','samsung-best-sd-memory-cards','samsung-best-security-cameras','samsung-best-smart-tvs','samsung-best-smartwatches','samsung-best-tablets','samsung-best-tablets-under-200','samsung-best-tvs','samsung-best-wireless-chargers')
and aa.id in (74131)
--and ts.sub1 not like '%group%'
--and ts.sub1 not in ('hotukdeals','savoo')
--and adc.DM_CLUSTER_ID not in (2001346202,2001346203,2001346200,2001346201,2001346205)
--and ts.sub2 in ('mercedes.com')
--and ts.sub3 = ''
--and ts.sub4 in ('desktop','mobile')
--and sub1 not in ((select DISTINCT ts.sub1 
--					from ampx.GLOBAL_BLACKLIST gb 
--					join ampx.TRAFFIC_SOURCE ts on ts.id = gb.TS_ID
--					where ts.PUBLISHER_ID in (12371) ))
--and ad.campaign_id in (217743)
--and sub2 ='rakuten'
--and level =4
order by 1
"""

ats_id = pd.read_sql(sql,db)

cluster_id = list(ats_id['DM_CLUSTER_ID'].unique())

print(len(cluster_id))

db.close()

In [ ]:
# cluster_id=[2000160427]
cluster_id

In [ ]:
# Set up dynamic and direct matches
file = open('experian_dynamic_remapping_04262024.sql','w') 

file.writelines('\nuse domainpub;\n')

for i in cluster_id :
    for j in ads:
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(j['ADGROUP_ID'],i,j['KEYWORD_ID']))
        file.writelines('\n')
#     file.writelines('\n\n')       
    
                    
#     break
    
file.close()

In [ ]:
# Set up dynamic and direct matches
file = open('skl_homedepotflooring_setup_20220719.sql','w') 

file.writelines('\nuse domainpub;\n')

for i in cluster_id :
    for j,k in zip(df['ADGROUP_ID'],df['KEYWORD_ID']):
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(j,i,k))
        file.writelines('\n')
#     file.writelines('\n\n')       
    
                    
#     break
    
file.close()

## Set up based on csv file

In [ ]:
import pandas as pd

In [ ]:
para = {'pub_id':[12513], 
        'ads': {},
        'sub1': ['1525078'],
        'sub2': ['casper'],
        'sub3': ['dynamiclink'],
        'sub4': ['mobile','desktop'],
        'account_id': [''],
        'ts_level_to-copy-ad':4,
        'ts_level_to-get-ts':4,
        'sub-value-to-copy-ad':'sub1',
        'sub-value-to-pass-ad':'sub1',        
       'ts_index':'BR_samsung_new_sub_structure_20201216.log',
       'cluster_sql':'./clusters/BR_samsung_new_sub_structure_update_20201216.sql'
       }

In [ ]:
df = pd.read_csv('samsung-br-setup-adding-more_2.csv')

In [ ]:
df.head()

In [ ]:
dic={}
sub1 = df['SUB1'].to_list()
for i,j in zip(sub1,df.iloc[:,4:].values):
    dic[i]=[]
    n=len(j)
    k=0
    while k<n:
        if j[k]==j[k]:
            dic[i].append([j[k],j[k+1]])
            k+=2
        else:
            k+=1

In [ ]:
para['sub1']=sub1

In [ ]:
para['ads']=dic

In [ ]:
sub1

In [ ]:
file = open(para['ts_index'],'w') 

for i in para['pub_id']:
    for j in para['sub1']:
            for k in para['sub2']:
                    for l in para['sub3']:
                            for m in para['sub4']:
                                line = str(i) + '|' + str(j) + '|' + str(k)+ '|' + str(l) + '|' + str(m)                           
                                file.writelines('%s\n'%line)
        
    
file.close()

In [ ]:
## Initial Setup
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=jzhu;PWD=%s'%pwd)

sql = """SELECT DISTINCT {0} as sub,
ats.ID FROM ampx.TRAFFIC_SOURCE ats 
WHERE ats.publisher_id IN ({1}) 
--and date(ats.CREATED_TIME_STAMP) = current_date
and ats.sub1 IN ({2}) 
and ats.sub2 in ({3})
and level = ({4})
order by 1
""".format(para['sub-value-to-pass-ad'].replace(',','||'),str(para['pub_id'])[1:-1],str(para['sub1'])[1:-1],
           str(para['sub2'])[1:-1],para['ts_level_to-get-ts'])

ats_id = pd.read_sql(sql,db)

ats_id.set_index('ID',inplace=True)

para['ats_id'] = ats_id.to_dict('index')

print(len(para['ats_id']))

db.close()

In [ ]:
para['ats_id']

In [ ]:
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=jzhu;PWD=%s'%pwd)

sql = """SELECT max(ID)+1 from domainpub.DM_CLUSTER"""

cluster_1st = db.cursor().execute(sql).fetchall()[0][0]

db.close()

In [ ]:
# cluster_1st = 2000160427
cluster_1st

In [ ]:
# Set up dynamic and direct matches
file = open(para['cluster_sql'],'w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for k in list(para['ats_id'].keys()):
    i = para['ats_id'][k]['sub']
    file.writelines('INSERT INTO DM_CLUSTER (ID, RESTRICTION_TYPE_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({}, 3, now(), now());'.format(cluster_1st))
    file.writelines('\n')

    file.writelines('INSERT INTO TS_CLUSTER (PUB_ATS_ID, CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP) VALUES({0}, {1}, now(), now());'.format(k,cluster_1st))
    file.writelines('\n')
     
    for j in para['ads'][i]:      
#         print(j)
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(str(int(j[0])),cluster_1st,str(int(j[1]))))
        file.writelines('\n')
    file.writelines('\n\n')       
    
    cluster_1st += 1
                    
#     break
    
file.close()

In [ ]:
# update and insert new asset
db = pyodbc.connect('DRIVER={Vertica};SERVER=w-vertica-read.sf.admarketplace.net;PORT=5433;\
                            DATABASE=amp;UID=jzhu;PWD=%s'%pwd)

sql = """SELECT distinct ats.sub1, tc.CLUSTER_ID as ID
from ampx.TRAFFIC_SOURCE ats 
join domainpub.TS_CLUSTER tc on tc.PUB_ATS_ID = ats.ID 
WHERE ats.publisher_id IN ({1}) 
and ats.sub1 IN ({2}) 
and ats.sub2 in ({3})
and level = ({4})
order by 1
""".format(para['sub-value-to-pass-ad'].replace(',','||'),str(para['pub_id'])[1:-1],str(para['sub1'])[1:-1],
           str(para['sub2'])[1:-1],para['ts_level_to-get-ts'])

ats_id = pd.read_sql(sql,db)

ats_id.set_index('ID',inplace=True)

para['clutser_id'] = ats_id.to_dict('index')

print(len(para['clutser_id']))

db.close()

In [ ]:
para['clutser_id']

In [ ]:
# Set up dynamic and direct matches
file = open(para['cluster_sql'],'w') 

file.writelines('\n\n-- CLUSTER CREATION --\n')
file.writelines('\nuse domainpub;\n')

for k in list(para['clutser_id'].keys()):
    i = para['clutser_id'][k]['sub1']    
    for j in para['ads'][i]:      
#         print(j)
        file.writelines('INSERT INTO ADGROUP_DM_CLUSTER (ADGROUP_ID, DM_CLUSTER_ID, CREATED_TIME_STAMP, MODIFIED_TIME_STAMP, KEYWORD_ID) VALUES({0}, {1}, now(), now(), {2});'.format(str(int(j[0])),k,str(int(j[1]))))
        file.writelines('\n')
#     file.writelines('\n')       
                    
#     break
    
file.close()

In [ ]:
str(int(dic['best-verizon-phones'][0][0]))

In [ ]:
dic['samsung-best-5g-phones']

In [ ]:
ip_lst = ['64.42.178.10','64.59.88.3','104.37.172.19','103.102.163.170','104.227.246.234','103.107.199.90','81.17.22.58','43.251.180.18','94.229.76.250','64.42.177.194','155.254.18.113','165.73.240.226','103.62.49.104','172.107.241.251','172.107.217.187','14.1.29.58','116.206.73.34','151.106.12.194','200.25.1.36','146.0.235.122','172.107.194.138','103.1.213.154','185.57.62.186','142.54.184.58','103.201.129.42','203.10.98.250','5.101.147.138','185.25.48.251','103.107.199.114','64.59.88.4','185.94.236.118','203.10.98.242','185.94.236.109','77.223.142.246']

In [ ]:
pub_lst = [11777,12243,12197,12329,12715]

In [ ]:
str_fmt = "CALL WHITELIST_IP_ADDRESS('{0}', {1}, 0, 'DB-10671',@WL_ID);"

In [ ]:
for i in pub_lst:
    for j in ip_lst:
        print(str_fmt.format(j,i))